In [1]:
%%capture
!pip install langchain langchain_groq langchain_community faiss-cpu

In [2]:
import os
os.environ["GROQ_API_KEY"]=""

In [3]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_groq import ChatGroq

physis_teacher="""
You are an Excelent physis teacher who has many year of expertise in this physis subject.

You can give ans to very complex question in detail and concise way but first give the direct answer then give explanation .
generate ans for this question:
{query}

"""
math_teacher="""
You are an Excelent math teacher who has many year of expertise in this math subject.
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question first give the direct ans then explain how this comes.
here is the question:
{query}

"""
bio_teacher="""
You are an Excelent Biology teacher who has many year of expertise in this biology subject.
You can give ans to very complex question in detail and concise way first give the direct answer in 30 to 40 words then give the explanation.
here is the question:
{query}
"""
programming_teacher = """
You are an excellent programming teacher with many years of expertise.
You can give answers to very complex questions in a detailed and concise way.
First, give the direct answer in 30 to 40 words, then give the explanation.
Here is the question:
{query}
"""

In [4]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

<ipython-input-4-c68e2a325111>:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
teacher_prompt=[math_teacher,bio_teacher,physis_teacher,programming_teacher]
prompt_embeddings = hf_embeddings.embed_documents(teacher_prompt)


llm=ChatGroq(temperature=0)
#routing query to prompt
def prompt_router(input):
    # Embed question
    query_embedding = hf_embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = teacher_prompt[similarity.argmax()]
    # Chosen prompt
    # print(f"most_similar is: {most_similar}")
    if most_similar==math_teacher:
      print("math_teacher is here : =>")
    elif most_similar==bio_teacher:
      print("bio teacher is here=>")
    elif most_similar==physis_teacher:
      print("physis teacher is here=>")
    else:
      print("programming teachere is here=>")


    return PromptTemplate.from_template(most_similar)


In [6]:
chain=(
    {"query":RunnablePassthrough()}

    |RunnableLambda(prompt_router)
    |llm
    |StrOutputParser()
)

In [8]:
chain.invoke("solve this 2+10")

math_teacher is here : =>


'The solution to the problem 2 + 10 is 12.\n\nTo solve this problem, you can follow these steps:\n\n1. Identify the problem: In this case, the problem is an addition problem with the numbers 2 and 10.\n2. Perform the addition: To do this, you can add the digits in each number together. In this case, you can add 2 + 10 to get 12.\n3. Check your work: To check your work, you can double-check the addition to make sure you got the correct answer. In this case, 2 + 10 equals 12, so the answer is correct.\n\nI hope this helps! Let me know if you have any other questions.'

In [9]:
chain.invoke("what is biology?")

bio teacher is here=>


'Biology is the scientific study of living organisms and their interactions with each other and their environment. It covers various disciplines, including genetics, evolution, cell biology, ecology, and molecular biology. Biologists aim to understand the fundamental principles that govern the functioning of all living systems.'